In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import numpy as np

import re
import random

In [2]:
df_action = pd.read_csv("action.csv", index_col = 0)
df_comedy = pd.read_csv("comedy.csv", index_col = 0)
df_drama = pd.read_csv("drama_no_duplicado.csv", index_col = 0)

Sacar todo el df drama 0(incluido) 8000 (SIN INCLUIR) registros

In [3]:
primeras_drama_8000_df = df_drama.values.tolist()[0:8000]

In [3]:
df1 = pd.read_csv("drama_imdb_803_pelis.csv", index_col = 0)
df2 = pd.read_csv("drama_imdb_803_3000_pelis.csv", index_col = 0)
df3 = pd.read_csv("drama_imdb_3000_3492_pelis.csv", index_col = 0)
df4 = pd.read_csv("drama_imdb_resto_pelis.csv", index_col = 0)

In [4]:
drama_df_0_a_7999 = pd.concat([df1, df2, df3, df4])

In [6]:
drama_df_0_a_7999.to_csv("TOTAL_SAIRA_drama_de_0_a_7999.csv")

In [5]:
len (drama_df_0_a_7999)

7999

In [11]:
len (df_action)

21414

In [9]:
len (df_comedy)

24000

In [10]:
len (df_drama)

41000

In [41]:
df_drama.count()

Tipo      41000
Título    41000
Año       40928
Mes       37018
ID        41000
dtype: int64

In [47]:
df_drama.drop_duplicates(subset=['ID']).to_csv('drama_no_duplicado.csv')

In [46]:
df_drama.groupby(['ID']).count()

,Tipo,Título,Año,Mes
ID,,,,
tt0011216,1,1,1,1
tt0062336,1,1,1,1
tt0069049,1,1,1,1
tt0079644,3,3,3,0
tt0083060,1,1,1,1
...,...,...,...,...
tt8679660,1,1,1,0
tt8810748,1,1,1,1
tt9389916,1,1,1,1


In [5]:
def datos_imdb (lista):
    # Inicializar navegador

    driver = webdriver.Chrome() 

    # URL IMbd para extraer la información solicitada. 

    url = "https://www.imdb.com/" 

    # Hacer una solicitud GET a la URL especificada.

    driver.get(url)  

    # Maximizar la ventana - ¡ojo! con las posiciones de los productos según se abra el navegador = separar inspeccion con dock side en los 3 puntitos. 

    driver.maximize_window()

    sleep(5)

    # Aceptar cookies

    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # Aceptar cookies

    # Creamos lista vacía para introducir el resultado de todas las peliculas con todos sus datos.

    #total_pelis = []

    # Iniciamos bucle for por cada código accediendo por su indice.
    
    
    for indice, peli in enumerate(lista):
        try:
            sleep(3)
            driver.find_element("css selector","#suggestion-search").send_keys(peli[4], Keys.ENTER)  # Se introduce cada código en el campo de búsqueda y accede a la busqueda de la info que le vamos a pedir. 
        except Exception as err:
            print(f"{err}: 404")
            driver.find_element("css selector","#error > div.error_message > a").click()
            sleep(3)

            driver.find_element("css selector","#suggestion-search").send_keys(peli[4], Keys.ENTER)
           
        pelicula = [] #Lista vacía para ir introduciendo todos los datos de cada pelicula
        
        #puntuación

        try:

            puntuacion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-69e49b85-1.llNLpA > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text.replace(",",".")
            
            pelicula.append(puntuacion)
        except: 
            
            pelicula.append("No tiene datos")

        # Ejecutamos Try/Except para obtener los datos de dirección, argumento, guionista y nombre de la película de la misma forma.
        # Se indica en el try/except varios selectores diferentes según localizados en la web en donde se pueda localizar cada dato. 
        # Con el except itera en varias opciones en el caso de que el primer try no haya funcionado , ejecutara el que encuentre, y sino imprime el mensaje de no hay datos. 

        # Dirección

        try:
            direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
            
            pelicula.append(direccion)

        except:

            try:
                direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
                
                pelicula.append(direccion)

            except:

                try:
                    direccion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.fVGakt > div > div.sc-491663c0-10.RRTJr > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(1) > div > ul > li > a").text
                    
                    pelicula.append(direccion)

                except:
                    
                    pelicula.append("No tiene datos")

            
        # Guionista

        try:
            guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div > ul > li > a").text
            patron = re.compile(r'([a-z])([A-Z])')
            # Usamos re.sub() para insertar una coma entre el nombre en minúsculas y la letra mayúscula
            resultado = re.sub(patron, r'\1, \2', guionista)
            
            pelicula.append(resultado)

        except:

            try:
                guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div > ul").text
                patron = re.compile(r'([a-z])([A-Z])')
                # Usamos re.sub() para insertar una coma entre el nombre en minúsculas y la letra mayúscula
                resultado = re.sub(patron, r'\1, \2', guionista)
                
                pelicula.append(resultado)

            except:

                try:
                    guionista = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.fVGakt > div > div.sc-491663c0-10.RRTJr > section > div.sc-69e49b85-3.dIOekc > div > ul > li:nth-child(2) > div").text
                    
                    pelicula.append(guionista)
                except:
                    
                    pelicula.append("No tiene datos")

        # Argumento
            
        try:
            
            argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p > span.sc-466bb6c-2.chnFO").text
            
            if argumento != "":

                pelicula.append(argumento)

            elif argumento == "":
                pelicula.append("No tiene argumento")
                
        except:
            
            try:
                argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.gEsAEH > div.sc-491663c0-6.eQRCDK > div.sc-491663c0-10.emoxHI > section > p > span.sc-466bb6c-2.chnFO").text
                
                if argumento != "":

                    pelicula.append(argumento)

                elif argumento == "":
                    pelicula.append("No tiene argumento") 

            except:
                
                pelicula.append("No tiene datos")


        # Duración (min)
        # 3 opciones de selector; 

        try:  
            try: # Primer selector, mayoría. 
            
                duracion_pre = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(3)").text
                duracion_str = duracion_pre.replace("h","").replace("min","").split() # Reemplazamos la h por vacio , y los minutos por espacio en blanco/vacio. 
                #print (duracion_str)  

                if len(duracion_str) == 2:                              # Primera condición ; si son 2 números, (hora y minutos) pasamos todo a minutos. 

                    duracion = int (duracion_str[0])*60 + int (duracion_str[1])
                    
                    pelicula.append(duracion)

                elif 'R' == duracion_str or "Not Rated" == duracion_str or "pg-13" == duracion_str or "g" == duracion_str or "pg" == duracion_str or "nc-17": # Segunda condición ; el anterior selector se utiliza para clasificación y la duración en otra parte. Seleccionamos nuevo selector a buscar.

                    duracion_pre = driver.find_element("css selector",'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(3)').text
                    duracion_str = duracion_pre.replace("h","").replace("min","").split()
                    #print("tiene R")
                    
                
                    if len(duracion_str) == 2:                                  
                        duracion = int(duracion_str[0])*60 + int (duracion_str[1])
                        
                        pelicula.append(duracion)

                    elif len(duracion_str) == 1 and "h" in duracion_pre:
                        duracion = int(duracion_str[0])*60
                        
                        pelicula.append(duracion)

                    else:
                        duracion = int (duracion_str[0])
                        
                        pelicula.append(duracion)            


                elif len(duracion_str) == 1 and "h" in duracion_pre:  # Si solo tiene un número y una h , lo pasamos a min. 
                        duracion = int(duracion_str[0])*60
                        
                        pelicula.append(duracion)

                else:                                                  # Directamente encontramos minutos.
                    duracion = int (duracion_str[0])
                    
                    pelicula.append(duracion)


            except:                                         # Tercer selector, por si las dos opciones primeras no funcionan. 

                duracion_pre = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul > li:nth-child(2)').text
                duracion_str = duracion_pre.replace("h","").replace("min","").split()
                
                if len(duracion_str) == 2:                                
                    duracion = int (duracion_str[0])*60 + int (duracion_str[1])
                    
                    pelicula.append(duracion)

                elif len(duracion_str) == 1 and "h" in duracion_pre:
                    duracion = int(duracion_str[0])*60
                    
                    pelicula.append(duracion)
                else:
                    duracion = int (duracion_str[0])
                    
                    pelicula.append(duracion)

        # Si no se cumplen ninguno de las condiciones anteriores, ejecuta no tiene datos. 
                    
        except:
            
            pelicula.append("No tiene datos")
            
        # Nombre de la película

        try:
            nombre_pelicula = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > h1 > span").text
            
            pelicula.append(nombre_pelicula)

        except:
            
            pelicula.append("No tiene datos")

        # Almacenamos en una variable , la conversión de la lista a tupla. 
            
        pelicula_tupla =  tuple(pelicula)
        total_pelis.append(pelicula_tupla)
        print(indice, pelicula_tupla)
        # Salimos a la pag.principal del navegador
        
        driver.back() 

    return total_pelis    # Impresion lista total 

In [6]:
total_pelis = []

In [ ]:
primeras_8000_drama_imdb = datos_imdb(primeras_drama_8000_df)

In [10]:
total_pelis[490]

('No tiene datos',
 'No tiene datos',
 'No tiene datos',
 'No tiene datos',
 'No tiene datos',
 'No tiene datos')

In [9]:
len(total_pelis)

4508

In [8]:
for pelii in total_pelis:
    if len(pelii) != 6:
        print("mal")

In [4]:
# Inicializar navegador
def datos_tomatoes (lista):

    driver = webdriver.Chrome() 

    # URL Google para extraer la información solicitada. 

    url = "https://www.google.com/" 

    # Hacer una solicitud GET a la URL especificada.

    driver.get(url)  

    # Maximizar la ventana

    driver.maximize_window()
    


    sleep(5)

    # Aceptar cookies

    driver.find_element("css selector", "#L2AGLb > div").click()

    # Creamos lista vacía para introducir el resultado de las puntuaciones (tomatometer) de cada pelicula.

    #total_puntuacion_tomatoes = []

    # Iniciamos bucle for por cada pelicula de los csv que tenemos accediendo por su indice.

    for indice,c in enumerate(lista): 
        

        sleep(random.randint(3,10))
        #creamos un string para pasarle a Google
        peticion = f"'Tomatometer de {c[1]} del año {c[2]}'"
        #incluimos el string en la barra de busqueda de Google
        driver.find_element("css selector","#APjFqb").send_keys(peticion, Keys.ENTER)  

        #puntuación (tomatometer)
        
        try:

            #creamos una variable con el nombre de la primera pelicula que aparece en Google
            nombre_peli_google = driver.find_element("css selector","#rso > div:nth-child(1) > div > div > div.kb0PBd.cvP2Ce.jGGQ5e > div > div > span > a > h3").text
            

            #si el nombre es igual al nombre de la pelicula que pedimos la añadimos a la lista
            if nombre_peli_google == c[1]:
                
                puntuacion = driver.find_element("css selector","#rso > div:nth-child(1) > div > div > div:nth-child(3) > div > span:nth-child(2)").text.replace("%","").replace("Valoración: ", "")
                total_puntuacion_tomatoes.append(puntuacion)
                print(puntuacion)

            else:
                try:
                    nombre_peli_google2 = driver.find_element("css selector","#rso > div:nth-child(2) > div > div > div.kb0PBd.cvP2Ce.jGGQ5e > div > div > span > a > h3").text

                    if nombre_peli_google2 == c[1]:
                
                        puntuacion = driver.find_element("css selector","#rso > div:nth-child(2) > div > div > div:nth-child(3) > div > span:nth-child(2)").text.replace("%","").replace("Valoración: ", "")
                        total_puntuacion_tomatoes.append(puntuacion)
                        print(puntuacion)
                    else:
                        total_puntuacion_tomatoes.append("No tiene datos")
                        print("No tiene datos")
                except:
                    total_puntuacion_tomatoes.append("No tiene datos")
                    print("No tiene datos")
        except: 

            total_puntuacion_tomatoes.append("No tiene datos")
            print("No tiene datos")
        
        print(indice, c[1], c[2], c[4])
        # Salimos a la pag.principal del navegador
        
        driver.back()

    return total_puntuacion_tomatoes    # Impresion lista total 

In [5]:
total_puntuacion_tomatoes = []

In [6]:
primeras_1000_drama_tomatoes = datos_tomatoes(primeras_drama_8000_df)

No tiene datos
0 Pál Adrienn 2011.0 tt0146592
No tiene datos
1 Oda az igazság 2010.0 tt0154039
No tiene datos
2 A zöld sárkány gyermekei 2011.0 tt0162942
No tiene datos
3 The Final Journey 2010.0 tt0230212
13
4 In My Sleep 2010.0 tt0326965
No tiene datos
5 This Wretched Life 2010.0 tt0331312
No tiene datos
6 Blind Sided 2010.0 tt0337882
No tiene datos
7 Anderson's Cross 2010.0 tt0393049
No tiene datos
8 Den milde smerte 2014.0 tt0396123
No tiene datos
9 Capture the Flag 2010.0 tt0402887
No tiene datos
10 Utomlennye solntsem 2 2010.0 tt0403645
0
11 Tekken 2010.0 tt0411951
42
12 Disconnect 2010.0 tt0419692
29
13 Milenge Milenge 2010.0 tt0443331
No tiene datos
14 Les 7 jours du talion 2010.0 tt0445054
No tiene datos
15 Vidas pequeñas 2011.0 tt0444531
No tiene datos
16 Dream Street 2010.0 tt0446686
65
17 Mad World 2010.0 tt0451108
No tiene datos
18 El hada buena - Una fábula peronista 2010.0 tt0454155
No tiene datos
19 Memorias del desarrollo 2011.0 tt0457591
No tiene datos
20 Drunkboat 20

In [ ]:
total_puntuacion_tomatoes

In [ ]:
#pasamos lista tomatoes a dataframe
df_drama_8000_tomatoes = pd.DataFrame(total_puntuacion_tomatoes)

In [ ]:
#guardamos df tomatoes en csv
df_drama_8000_tomatoes.to_csv("TOTAL_SAIRA_tomatoes_drama_de_0_a_7999.csv")

In [ ]:
#traemos csv imdb
df_imdb = pd.read_csv("TOTAL_SAIRA_imdb_drama_de_0_a_7999.csv", index_col = 0)

In [ ]:
#pasamos csv imdb a lista
primeras_drama_8000_imdb = df_imdb.values.tolist()

In [ ]:
#vemos que tenga 7999 datos
len(primeras_drama_8000_imdb)

In [ ]:
#traemos csv tomatoes
df_tomatoes = pd.read_csv("TOTAL_SAIRA_tomatoes_drama_de_0_a_7999.csv", index_col = 0)

In [ ]:
#pasamos csv tomatoes a lista
primeras_drama_8000_tomatoes = df_tomatoes.values.tolist()

In [ ]:
#vemos que tenga 7999 datos
len(primeras_drama_8000_tomatoes)

In [30]:
# Incluir puntuacion tomatoes en la tupla de imdb
def fase2_unificada (lista_imdb, lista_tomatoes, lista_df):

    lista_final_fase2 = []

    for indice, tupla in enumerate(lista_imdb):
        lista_incluir_tomatometer = list(tupla)
        lista_incluir_tomatometer.append(lista_df[indice][5]) ## QUITAR!! incluyo codigo imdb por si hay que revisar algo
        lista_incluir_tomatometer.insert(1, lista_tomatoes[indice])
        tupla_completa = tuple(lista_incluir_tomatometer)
        lista_final_fase2.append(tupla_completa)
        

    return lista_final_fase2

In [31]:
primeras_1000_accion_todo = fase2_unificada(primeras_drama_8000_imdb, primeras_drama_8000_tomatoes, primeras_drama_8000_df)

In [ ]:
#encabezado = ["Puntuacion IMDB", "Puntuacion Rotten Tomatoes", "Direccion", "Guionistas", "Argumento", "Duracion", "Nombre"]

In [10]:
df_drama_imdb_resto = pd.DataFrame(total_pelis)

df_drama_imdb_resto

,0,1,2,3,4,5
0,5.8,Taylor Hackford,Art Linson,Un vistazo a la vida de un malhablado y enveje...,120,The Comedian
1,5.7,Warren Beatty,Warren Beatty,La historia de un amor poco convencional entre...,127,La excepción a la regla
2,No tiene datos,Marko Sanginetto,Brian Carlton,"To escape his memories, a man disappears into ...",No tiene datos,Mud Man
3,7.2,Duane Andersen,Duane Andersen,A superhero loses his powers shortly after tur...,80,Superpowerless
4,6.2,Susanna White,John le Carré,Una pareja se ve arrastrada a los planes de de...,108,Un traidor como los nuestros
...,...,...,...,...,...,...
4503,6.2,Jane Cusumano,Jane Cusumano,Despite different backgrounds and disapproving...,105,What Matters Most
4504,5.8,Dani Minnick,Dani Minnick,Set in California the story traces the intense...,94,Falling Like This
4505,6.0,Stéphane Giusti,"Raphaëlle Desplechin, Stéphane Giusti","Nella and Orfeo, communists, flee Tuscany with...",103,Bella ciao
4506,6.8,Deborah Attoinese,Deborah Attoinese,Three runaway girls hijack a car.,90,Zoe


In [11]:
df_drama_imdb_resto.to_csv("drama_imdb_resto_pelis.csv")